# Sequential Spin Constraints (Stern–Gerlach)

By simulating back-to-back Stern–Gerlach measurements, we show how contextual constraints govern which correlations survive. The notebook highlights that “collapse” is simply the system reorganising under a new constraint in the present. It illustrates the systemic thesis: the measured spin, magnets, and detectors act as one whole whose lawful behaviour is validated through current correlations, not inferred histories.

In [1]:
import numpy as np

from wavefunction_now.measurement import chi_squared_gof, ks_goodness_of_fit

In [2]:
rng = np.random.default_rng(123)

ket_up_z = np.array([1.0, 0.0], dtype=complex)
ket_down_z = np.array([0.0, 1.0], dtype=complex)
ket_plus_x = (ket_up_z + ket_down_z) / np.sqrt(2)

proj_up_z = np.outer(ket_up_z, ket_up_z.conj())
proj_down_z = np.outer(ket_down_z, ket_down_z.conj())
proj_plus_x = 0.5 * np.array([[1, 1], [1, 1]], dtype=complex)
proj_minus_x = 0.5 * np.array([[1, -1], [-1, 1]], dtype=complex)

projectors_z = [proj_up_z, proj_down_z]
projectors_x = [proj_plus_x, proj_minus_x]

def projective_measure(state, projectors):
    probs = np.array([np.real(state.conj().T @ P @ state) for P in projectors])
    probs = np.clip(probs, 0.0, 1.0)
    probs = probs / probs.sum()
    outcome = rng.choice(len(projectors), p=probs)
    post = projectors[outcome] @ state
    post_norm = np.linalg.norm(post)
    if post_norm == 0:
        raise RuntimeError('Projection annihilated the state.')
    return outcome, post / post_norm

def run_sequence(num_events, second_axis):
    if second_axis == 'z':
        projectors_second = projectors_z
        expected = np.array([0.5, 0.5])
        events = np.zeros(num_events, dtype=int)
        counts = np.zeros(2, dtype=int)
    elif second_axis == 'x':
        projectors_second = projectors_x
        expected = np.full(4, 0.25)
        events = np.zeros(num_events, dtype=int)
        counts = np.zeros(4, dtype=int)
    else:
        raise ValueError('second_axis must be "z" or "x"')

    for i in range(num_events):
        state = ket_plus_x.copy()
        first_outcome, state = projective_measure(state, projectors_z)
        second_outcome, state = projective_measure(state, projectors_second)
        if second_axis == 'z':
            idx = first_outcome
        else:
            idx = first_outcome * 2 + second_outcome
        counts[idx] += 1
        events[i] = idx

    return counts, events, expected

In [3]:
counts_same, events_same, expected_same = run_sequence(5000, 'z')
chi2_same, p_same = chi_squared_gof(expected_same, counts_same)
d_same, ks_p_same = ks_goodness_of_fit(expected_same, events_same)

print('Z then Z (matching axis)')
print('Counts:', counts_same)
print(f'Chi-squared statistic: {chi2_same:.3f}, p-value: {p_same:.3f}')
print(f'KS statistic: {d_same:.3f}, p-value: {ks_p_same:.3f}')

Z then Z (matching axis)
Counts: [2525 2475]
Chi-squared statistic: 0.500, p-value: 0.480
KS statistic: 0.005, p-value: 0.776


In [5]:
counts_cross, events_cross, expected_cross = run_sequence(5000, 'x')
chi2_cross, p_cross = chi_squared_gof(expected_cross, counts_cross)
d_cross, ks_p_cross = ks_goodness_of_fit(expected_cross, events_cross)

print('\nZ then X (orthogonal axis)')
print('Counts:', counts_cross)
print(f'Chi-squared statistic: {chi2_cross:.3f}, p-value: {p_cross:.3f}')
print(f'KS statistic: {d_cross:.3f}, p-value: {ks_p_cross:.3f}')


Z then X (orthogonal axis)
Counts: [1240 1242 1263 1255]
Chi-squared statistic: 0.286, p-value: 0.963
KS statistic: 0.004, p-value: 0.876


Sequential measurements along the same axis retain the initial outcome, whereas switching to an orthogonal axis restores a uniform distribution. Both behaviours fall naturally out of present-time projective updates and are captured by the statistical tests.